In [ ]:
import sys
sys.path.insert(0, "/home/jens/PycharmProjects/arbfree-dyn-ns")

from IPython.display import clear_output
import ipywidgets as widgets
from pandas import Timestamp
import statsmodels.api as sm
import datetime
import re

from arbfree_dyn_ns import *

In [ ]:
RESULTS_BASE_PATH = PKL_BASE_PATH

In [ ]:
results = { p.name: pd.read_pickle(p)
         for p in sorted(RESULTS_BASE_PATH.glob("main*.pkl"))
           if file_name_to_datetime(p) > datetime.datetime(2024, 1, 2, 14, 38)
              #and ("Wkly" if frequency_weekly else "Mthly") in p.name and cutoff.strftime("%y%m%d") in p.name
          }

In [ ]:
def i_str(i, n):
    # test: [[i_str(i, n) for i in range(n)] for n in [3, 5]]
    return ("level" if i == 0
             else "slope1" if i == 1
             else "slope2" if i == 2 and n == 5
             else "curvature1" if i == 2
             else "curvature1" if i == 3
             else "curvature2")

results_dissected = {}
for key, result in results.items():
    results_dissected[key] = dict(zip(("optres", "H_opt", "K_opt", "Sigma_opt", "lambda_opt", "lambda_svensson_opt", "theta_opt", 
                "theta_xs", "K_xs", "H_xs", "Sigma_xs", "loglikelihood_xs", "lambda_xs", "lambda_svensson_xs", "n_ev"), result))
    results_dissected[key].update(dict(results_dissected[key]["optres"]))
    results_dissected[key]["yield_adjustment_term_included"] = "True" in key
    dissected_file_name = dissect_file_name(key)
    if dissected_file_name is not None:
        results_dissected[key].update(dissected_file_name)
    for k in ("H_opt", "K_opt", "Sigma_opt", "theta_opt"):
        if k != "theta_opt":
            results_dissected[key][k] = np.diagonal(results_dissected[key][k])
        n = len(results_dissected[key][k])
        results_dissected[key]["n"] = n
        if n in (3, 5):
            for i in range(n):
                results_dissected[key][k + "_" + i_str(i, n)] = results_dissected[key][k][i]
    #print(results_dissected[key].keys())
    results_dissected[key] = pd.Series(results_dissected[key])
results_dissected = pd.DataFrame(results_dissected).T

results_dissected = results_dissected.join(results_dissected["yields"].apply(
    lambda df: pd.Series({"timeframe_start": df.index.min() if hasattr(df, "index") else pd.Timestamp("1970-01-01"),
                          "timeframe_end": df.index.max() if hasattr(df, "index") else pd.Timestamp("1970-01-01")})),
                       how="left")

In [ ]:
columns_of_interest = ['fun', 'message', 'n', 'yield_adjustment_term_included', 'timeframe_start',
       'timeframe_end', 'data_source', 'frequency', 'K_opt_level', 'K_opt_slope1', 'K_opt_slope2', 'K_opt_curvature1', 'K_opt_curvature2',
                       'Sigma_opt_level', 'Sigma_opt_slope1', 'Sigma_opt_slope2', 'Sigma_opt_curvature1', 'Sigma_opt_curvature2',
                       'theta_opt_level', 'theta_opt_slope1', 'theta_opt_slope2', 'theta_opt_curvature1', 'theta_opt_curvature2',
        'lambda_opt',
       'lambda_svensson_opt']

In [ ]:
results_dissected[columns_of_interest]

In [ ]:
results_dissected[columns_of_interest].to_excel(RESULTS_BASE_PATH / "results_dissected20240103.xlsx", freeze_panes=(1, 1))

In [ ]:
#results_dissected[columns_of_interest].to_latex()

In [ ]:
@widgets.interact(key=widgets.Dropdown(options=results.keys(),
                                      layout={'width': 'max-content'}))
def f(key):
    optres, H_opt, K_opt, Sigma_opt, lambda_opt, lambda_svensson_opt, theta_opt, \
            theta_xs, K_xs, H_xs, Sigma_xs, loglikelihood_xs, lambda_xs, lambda_svensson_xs, n_ev = results[key]
    display(optres)
    print("\n\n")
    
    print("jac     =", optres.jac, end="\n\n")
    print("H       =", np.diagonal(H_opt), end="\n\n")
    print("K       =", np.diagonal(K_opt), end="\n\n")
    print("Sigma   =", np.diagonal(Sigma_opt), end="\n\n")
    print("lambda1 =", lambda_opt, end="\n\n")
    print("lambda2 =", lambda_svensson_opt, end="\n\n")
    print("theta   =", theta_opt, end="\n\n")

In [ ]:
squared_error_described_apriori = {}
error_described_apriori = {}
squared_error_described_apost = {}
error_described_apost = {}

def run_kalman_for_opt_params(key, silent=False, save_plots=False):
    #print(key)
    if "TEST" in key:
        yields = afgns_data()[1]
        time_step_size = 1/12
    else:
        gd = dissect_file_name(key)
        yields = gd["yields"]
        time_step_size = 7/365.2425 if gd["frequency"] == "Wkly" else 1/12
    grid = yields.columns.rename("tenor")
    if not silent:
        print(grid)

    
    result = results[key]
    optres, H_opt, K_opt, Sigma_opt, lambda_opt, lambda_svensson_opt, theta_opt, \
                theta_xs, K_xs, H_xs, Sigma_xs, loglikelihood_xs, lambda_xs, lambda_svensson_xs, n_ev = result

    B_opt = B_matrix(lambda_opt, grid, lambda_svensson_value=lambda_svensson_opt)
    B_for_optimization = B_opt.to_numpy()
    if "True" in key:
        c_vector = C_vector(lambda_opt, grid, K_times_theta=None, Sigma=Sigma_opt, lambda_svensson_value=lambda_svensson_opt)
        c_vector = c_vector.to_numpy()
    else:
        c_vector = None
    
    
    A_opt = scipy.linalg.expm(-time_step_size * K_opt)
    Q_opt = compute_Q(K_opt, Sigma_opt, time_step_size)

  
    kalman_res = kalman_filter(yields, theta_opt, A_opt, H_opt, Q_opt, B=B_for_optimization,
                            exclude_first_observations_for_loglikelihood=min(10, .05 * len(yields)),
                            c=c_vector)  # , no_tqdm=no_tqdm)[1]
    x, loglikelihood, loglikelihood_contribution, P, F, v, x_next = kalman_res
    
    if not silent:
        print(loglikelihood)
        print(theta_opt)
        ax = pd.DataFrame(v, index=x.index, columns=grid).iloc[2:].plot()
        if save_plots:
            ax.get_figure().savefig(RESULTS_BASE_PATH / (key + "errors.svg"))
    if key not in squared_error_described_apost.keys():
        xtt = np.array([xtt_numba(x.iloc[i].to_numpy(), P[i], B_for_optimization, v[i], np.linalg.inv(F[i]))
                    for i in range(len(yields))], dtype=float)
        vtt = np.array([y.to_numpy() - B_for_optimization @ xtt_ for (i, y), xtt_ in zip(yields.iterrows(), xtt)],
                      dtype=float)
        if c_vector is not None:
            vtt = np.array([vtt_ - c_vector for vtt_ in vtt], dtype=float)
        
        for the_v, (squared_error_described, error_described) in ((v, (squared_error_described_apriori, error_described_apriori)),
                                                                  (vtt, (squared_error_described_apost, error_described_apost))):
            v_mat = pd.DataFrame(the_v, index=x.index, columns=grid).iloc[2:]
            error_described[key] = v_mat.describe()
            squared_error_described[key] = (v_mat ** 2).describe()
    if not silent:
        rename_dict = dict(enumerate(["level", "slope", "curvature"])) if x.shape[1] == 3 \
                        else dict(enumerate(["level", "slope1", "slope2", "curvature1", "curvature2"]))
        ax = x.rename(rename_dict, axis=1).plot(style=["-", "--", ":"] if x.shape[1] == 3 else ["-", "--", "--", ":", ":"])
        if save_plots:
            ax.get_figure().savefig(RESULTS_BASE_PATH / (key + "X.svg"))
        if c_vector is not None:
            display(pd.Series(c_vector, index=grid))
    #(B_opt @ theta_opt).plot()


widgets.interact(key=widgets.Dropdown(options=results.keys(),
                                      layout={'width': 'max-content'}))(run_kalman_for_opt_params);

In [ ]:
# generate svgs
for k in results.keys():
    if "TEST" not in k and k not in squared_error_described_apriori.keys():
        run_kalman_for_opt_params(k, silent=False, save_plots=True)

In [ ]:
# fill squared_error_described
for k in results.keys():
    if "TEST" not in k and k not in squared_error_described_apriori.keys():
        run_kalman_for_opt_params(k, silent=True)

In [ ]:
rmse_by_tenor = { key: pd.concat(squared_error_described_apriori if key == "apriori" else squared_error_described_apost)\
                       .xs("mean", level=1) ** (1/2) * 1e4
                 for key in ("apost", "apriori")}

me_by_tenor = { key: pd.concat(error_described_apriori if key == "apriori" else error_described_apost)\
                       .xs("mean", level=1) * 1e4
                 for key in ("apost", "apriori")}

rmse = { key: pd.concat(squared_error_described_apriori if key == "apriori" else squared_error_described_apost)\
                       .xs("mean", level=1).mean(axis=1) ** (1/2) * 1e4
                 for key in ("apost", "apriori")}

me = { key: pd.concat(error_described_apriori if key == "apriori" else error_described_apost)\
                       .xs("mean", level=1).mean(axis=1) * 1e4
                 for key in ("apost", "apriori")}

In [ ]:
errors = pd.concat({"rmse": pd.concat(rmse_by_tenor).join(pd.concat(rmse).rename("total")),
                    "me":   pd.concat(me_by_tenor  ).join(pd.concat(me  ).rename("total"))}).swaplevel(0,2).swaplevel(1,2)
errors.index.rename(["pkl_name", "error_type", "knowledge"], inplace=True)
errors = results_dissected[["n", "yield_adjustment_term_included", "timeframe_start", "timeframe_end", "data_source", "frequency"]].merge(
    errors, right_on="pkl_name", left_index=True, how="right")

In [ ]:
errors.to_excel(RESULTS_BASE_PATH / "errors20240103.xlsx", freeze_panes=(1, 1))

# LaTeX Output

In [ ]:
precise_timeframes = errors[["data_source", "frequency", "timeframe_start", "timeframe_end"]].drop_duplicates().reset_index(drop=True)

In [ ]:
index_cols = ['timeframe', 'frequency', 'n',
              'yield_adjustment_term_included', 
              'data_source'
             ]

errors_latex = errors.copy().droplevel(level=0).reset_index()
errors_latex["timeframe"] = errors_latex["timeframe_start"].dt.strftime("%y") + "--" + errors_latex["timeframe_end"].dt.strftime("%y")
errors_latex.sort_values(["data_source", "frequency", "timeframe_end", "timeframe_start",
                          "n", "yield_adjustment_term_included"], inplace=True)
errors_latex = errors_latex[errors_latex["timeframe_end"].dt.year != 2008]
errors_latex = errors_latex.drop(["timeframe_start", "timeframe_end"], axis=1).set_index(index_cols)
errors_latex = errors_latex[["error_type", "knowledge", 1, 2, 3, 5, 10, 15, 20, 25, "total"]]

results_dissected_latex = results_dissected.loc[~results_dissected["data_source"].isna(), columns_of_interest].copy()\
                        .reset_index(drop=True)\
                        .drop(["message"], axis=1).rename({"fun": "loglikelihood",
                                                      }, axis=1)
results_dissected_latex["loglikelihood"] = results_dissected_latex["loglikelihood"].abs()
results_dissected_latex["timeframe"] = results_dissected_latex["timeframe_start"].dt.strftime("%y") + "--" + results_dissected_latex["timeframe_end"].dt.strftime("%y")
results_dissected_latex.sort_values(["data_source", "frequency", "timeframe_end", "timeframe_start",
                                     "n", "yield_adjustment_term_included"], inplace=True)
results_dissected_latex = results_dissected_latex[results_dissected_latex["timeframe_end"].dt.year != 2008]
results_dissected_latex = results_dissected_latex.drop(["timeframe_start", "timeframe_end"], axis=1).set_index(index_cols)

formatters = {column_name: f"{{:0.{digits}f}}".format
                for column_name, digits in 
                (("loglikelihood", 1),
                 ("K_opt_slope1", 2),
                 ("K_opt_slope2", 2),
                 ("K_opt_level", 2),
                 ("K_opt_curvature1", 2),
                 ("K_opt_curvature2", 2),
                 ("Sigma_opt_level", 4),
                 ("Sigma_opt_slope1", 4),
                 ("Sigma_opt_slope2", 4),
                 ("Sigma_opt_curvature1", 4),
                 ("Sigma_opt_curvature2", 4),
                 ("theta_opt_level", 4),
                 ("theta_opt_slope1", 4),
                 ("theta_opt_slope2", 4),
                 ("theta_opt_curvature1", 4),
                 ("theta_opt_curvature2", 4),
                 ("lambda_opt", 4),
                 ("lambda_svensson_opt", 4),
                )}

In [ ]:
errors_latex_sub = {(error_type, knowledge):
        errors_latex.query(f"error_type == '{error_type}' and knowledge == '{knowledge}'").drop(["error_type", "knowledge"], axis=1)
    for error_type, knowledge in errors_latex[["error_type", "knowledge"]].drop_duplicates().values }

In [ ]:
for (error_type, knowledge), df in errors_latex_sub.items():
    for data_source in ("lw", "buba"):
        df2 = df.query(f"data_source=='{data_source}'").copy()
        df2 = df2.droplevel("frequency" if data_source == "lw" else "timeframe").droplevel("data_source")
        out = df2.to_latex(na_rep="---", sparsify=False, float_format="%.2f" if error_type != "me" else "%.2f")\
                .replace(" 00:00:00", "")\
                .replace(" buba ", " BuBa ")\
                .replace(" lw ", " Liu-Wu ")\
                .replace("Mthly ", "M ")\
                .replace("Wkly ", "W ")\
                .replace(" False ", " no ")\
                .replace(" True ", " yes ")\
                .replace(".000000", "")
        with open(RESULTS_BASE_PATH / f"errors_{error_type}_{knowledge}__{data_source}.inc", "w") as f:
            print(out, file=f)        

In [ ]:
results_dissected_subset_definitions = {'loglikelihood_and_lambda': ['loglikelihood', "lambda_opt", "lambda_svensson_opt"],
                                        'K': [c for c in results_dissected_latex.columns if c.startswith("K_")],
                                        'Sigma': [c for c in results_dissected_latex.columns if c.startswith("Sigma_")],
                                        'theta': [c for c in results_dissected_latex.columns if c.startswith("theta_")],
                                       }

results_dissected_latex_sub = { subset_name: results_dissected_latex[subset_columns]
                               for subset_name, subset_columns in results_dissected_subset_definitions.items()}

In [ ]:
out = results_dissected_latex.to_latex(formatters=formatters, na_rep="---", sparsify=False)\
    .replace(" 00:00:00", "")\
    .replace(" buba ", " BuBa ")\
    .replace(" lw ", " Liu-Wu ")\
    .replace(" Mthly ", " M ")\
    .replace(" Wkly ", " W ")\
    .replace(" False ", " no ")\
    .replace(" True ", " yes ")
with open(RESULTS_BASE_PATH / "results_dissected.inc", "w") as f:
    print(out, file=f)

for subset_name, df in results_dissected_latex_sub.items():
    for data_source in ("lw", "buba"):
        df2 = df.query(f"data_source=='{data_source}'").copy()
        df2 = df2.droplevel("frequency" if data_source == "lw" else "timeframe").droplevel("data_source")
        out = df2.to_latex(formatters=formatters, na_rep="---", sparsify=False)\
        .replace(" 00:00:00", "")\
        .replace(" buba ", " BuBa ")\
        .replace(" lw ", " Liu-Wu ")\
        .replace("Mthly ", "M ")\
        .replace("Wkly ", "W ")\
        .replace(" False ", " no ")\
        .replace(" True ", " yes ")
        with open(RESULTS_BASE_PATH / f"results_dissected_{subset_name}__{data_source}.inc", "w") as f:
            print(out, file=f)   

In [ ]:
out = precise_timeframes.to_latex(index=False)\
    .replace(" 00:00:00", "")\
    .replace(" buba ", " BuBa ")\
    .replace(" lw ", " Liu-Wu ")\
    .replace("Mthly ", "M ")\
    .replace("Wkly ", "W ")\
    .replace(" False ", " no ")\
    .replace(" True ", " yes ")
with open(RESULTS_BASE_PATH / "precise_timeframes.inc", "w") as f:
    print(out, file=f)

# C

In [ ]:
def c_opt(key, save_plot=False):
    if "TEST" in key:
        yields = afgns_data()[1]
        time_step_size = 1/12
    else:
        gd = dissect_file_name(key)
        yields = gd["yields"]
        time_step_size = 7/365.2425 if gd["frequency"] == "Wkly" else 1/12
    grid = yields.columns
    result = results[key]
    optres, H_opt, K_opt, Sigma_opt, lambda_opt, lambda_svensson_opt, theta_opt, \
                theta_xs, K_xs, H_xs, Sigma_xs, loglikelihood_xs, lambda_xs, lambda_svensson_xs, n_ev = result

    B_opt = B_matrix(lambda_opt, grid, lambda_svensson_value=lambda_svensson_opt)
    B_for_optimization = B_opt.to_numpy()
    c = C_vector(lambda_opt, grid, K_times_theta=None, Sigma=Sigma_opt, lambda_svensson_value=lambda_svensson_opt)
    c.index.rename("tenor", inplace=True)
    c = c.rename("yield adjustment term -C(tenor)/tenor")
    if save_plot:
        c.plot(marker="o").get_figure().savefig(RESULTS_BASE_PATH / (key + "C.svg"))
        plt.close()
    return c

In [ ]:
for key in results.keys():
    c_opt(key, save_plot=True)

In [ ]:
c_opt(next(iter(results.keys()))).plot(marker="o")